In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import roots_legendre
from tqdm import tqdm

import mu2

import constants as const
from interaction import potential
from counterterm import local_LO_counterterm
from mottmassey import mm_potential

gr = 4/3
h = 3

plt.style.use('science')

In [2]:
lo_rg_flow = np.loadtxt('rgflow_local_lo_a0_fix.txt')
nlo_rg_flow = np.loadtxt('rgflow_local_nlo_a0_B6_fix.txt')

ell = 0

In [3]:
rmesh = mu2.Mesh(0, 10*const.BETA4, 2000)
systems = []
for (Ri, _) in tqdm(lo_rg_flow):
    qmesh = mu2.Mesh(0, 20*2/Ri, 300, log=False)
    x = mu2.LocalCounterterm(local_LO_counterterm, lambda r, R: 0, rmesh, qmesh, Ri, ell)
    
    systems.append(mu2.System(mu2.Interaction(potential, x, rmesh), const.MU, ell))

100%|██████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:48<00:00,  4.57s/it]


In [4]:
ks = np.linspace(0.01/const.BETA4, 40/const.BETA4, 100)
# ks = np.linspace(0.01/const.BETA4, 1/1.5, 50)
deltas_lo = []

for i in tqdm(range(len(systems))):
    s = systems[i]
    gi = lo_rg_flow[i, 1]
    kcd = s.kcotd_gen_fast(ks, gi, 0)
    deltas_lo.append(np.arctan(ks/kcd))
#     kcds_lo.append(np.array([
#         mu2.cc.cscatter.kcotdelta_py(k, s.v_tilde + s.interaction.counterterm.gen(gi, 0),
#                                      s.q, s.wq, 20*2/s.r_c, ell, 2*const.MU) for k in ks]))

deltas_lo = np.array(deltas_lo)

100%|██████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:19<00:00,  1.60s/it]


In [5]:
np.savetxt('phase_shifts_lo.txt', np.vstack((ks, deltas_lo)).T)

In [6]:
deltas_nlo = []
for i in tqdm(range(len(systems))):
    s = systems[i]
    glo, gnlo = nlo_rg_flow[i, 1:]
    xterm = s.interaction.counterterm.gen(1, 0)
    kcd = np.array([
        mu2.cc.cscatter.kcotdelta_py(k, s.v_tilde + glo*xterm + gnlo*k**2/(2*const.MU)*xterm, 
                                     s.q, s.wq, 20*2/s.r_c, ell, 2*const.MU) for k in ks
    ])
    deltas_nlo.append(np.arctan(ks/kcd))

deltas_nlo = np.array(deltas_nlo)

100%|██████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:20<00:00,  1.61s/it]


In [7]:
np.savetxt('phase_shifts_nlo.txt', np.vstack((ks, deltas_nlo)).T)

In [12]:
mm = mu2.System(
    mu2.Interaction(lambda r, R: mm_potential(r),
        mu2.LocalCounterterm(lambda r, R: 0, lambda r, R: 0, rmesh, mu2.Mesh(0, 20*2/1.5, 300), 1.5, 0),
        rmesh
    ), const.MU, 0
)

kcds_mm = np.array([mu2.cc.cscatter.kcotdelta_py(k, mm.v_tilde, mm.q, mm.wq, 20*2/1.5, 0, 2*const.MU) for k in ks])
deltas_mm = np.arctan(ks/kcds_mm)

In [15]:
np.savetxt('phase_shifts_mott_massey.txt', np.vstack((ks, deltas_mm)).T)